<a href="https://colab.research.google.com/github/bubriks/ID2223/blob/main/Lab2/s-bert_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB-2 - Paolo Teta & Ralfs Zangis
---
useful:
https://www.sbert.net/examples/training/sts/README.html#training-data

## Setup Colab (do NOT execute)

In [4]:
# from google.colab import drive
# import os

# !pip install pyspark
# !pip install --upgrade tensorflow_hub
# !pip install --upgrade tensorflow_text

# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *
# from pyspark.sql.types import *

# spark = SparkSession.builder.getOrCreate()

# drive.mount('/content/gdrive')
# # os.mkdir("/content/gdrive/MyDrive/ID2223-repo/")

# # os.chdir("/content/gdrive/MyDrive/ID2223-repo/")
# # !git clone https://github.com/bubriks/ID2223.git

# os.chdir("/content/gdrive/MyDrive/ID2223-repo/ID2223/Lab2/")
# !git pull

# # ------------------------------ #

# schema = StructType([
#     StructField("genre", StringType(), True),
#     StructField("filename", StringType(), True),
#     StructField("year", StringType(), True),
#     StructField("year_id", IntegerType(), True),
#     StructField("score", FloatType(), True),
#     StructField("sentence1", StringType(), True),
#     StructField("sentence2", StringType(), True)])

# train = spark.read.csv("/content/gdrive/MyDrive/ID2223-repo/ID2223/Lab2/stsbenchmark/sts-test.csv", sep ='\t', header=False, schema=schema)
# test = spark.read.csv("/content/gdrive/MyDrive/ID2223-repo/ID2223/Lab2/stsbenchmark/sts-test.csv", sep ='\t', header=False, schema=schema)
# dev = spark.read.csv("/content/gdrive/MyDrive/ID2223-repo/ID2223/Lab2/stsbenchmark/sts-dev.csv", sep ='\t', header=False, schema=schema)

# START

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

In [6]:
# !pip install --upgrade tensorflow_hub
# !pip install --upgrade tensorflow_text

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import functools

import numpy as np

In [11]:
schema = StructType([
    StructField("genre", StringType(), True),
    StructField("filename", StringType(), True),
    StructField("year", StringType(), True),
    StructField("year_id", IntegerType(), True),
    StructField("score", FloatType(), True),
    StructField("sentence1", StringType(), True),
    StructField("sentence2", StringType(), True)])

train = spark.read.csv("stsbenchmark/sts-train.csv", sep ='\t', header=False, schema=schema)
test = spark.read.csv("stsbenchmark/sts-test.csv", sep ='\t', header=False, schema=schema)
dev = spark.read.csv("stsbenchmark/sts-dev.csv", sep ='\t', header=False, schema=schema)

train.show()
test.show()
dev.show()

+-------------+--------+--------+-------+-----+--------------------+--------------------+
|        genre|filename|    year|year_id|score|           sentence1|           sentence2|
+-------------+--------+--------+-------+-----+--------------------+--------------------+
|main-captions|  MSRvid|2012test|     24|  2.5|A girl is styling...|A girl is brushin...|
|main-captions|  MSRvid|2012test|     33|  3.6|A group of men pl...|A group of boys a...|
|main-captions|  MSRvid|2012test|     45|  5.0|One woman is meas...|A woman measures ...|
|main-captions|  MSRvid|2012test|     63|  4.2|A man is cutting ...|A man is slicing ...|
|main-captions|  MSRvid|2012test|     66|  1.5|A man is playing ...|A man is playing ...|
|main-captions|  MSRvid|2012test|     74|  1.8|A woman is cuttin...|A woman is cuttin...|
|main-captions|  MSRvid|2012test|     76|  3.5|A man is riding a...|A man is riding a...|
|main-captions|  MSRvid|2012test|     82|  2.2|A man is playing ...|A man is playing ...|
|main-capt

## Normalize score column

In [7]:
train = train.withColumn("score", col("score")/2.5-1)
test = test.withColumn("score", col("score")/2.5-1)
dev = dev.withColumn("score", col("score")/2.5-1)

dev.select("score").describe().show()

+-------+--------------------+
|summary|               score|
+-------+--------------------+
|  count|                1500|
|   mean|-0.05443697837591158|
| stddev|  0.6001942581590352|
|    min|                -1.0|
|    max|                 1.0|
+-------+--------------------+



In [8]:
text_test = ['this is such an amazing movie!']

tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

text_preprocessed = preprocessing_layer(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [9]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

bert_results = encoder(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"][0].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.7626282   0.9928099  -0.18611862  0.3667383   0.15233758  0.655044
  0.9681154  -0.94862705  0.0021616  -0.9877732   0.06842764 -0.97630596]
Sequence Outputs Shape:(128, 512)
Sequence Outputs Values:[[-0.28946292  0.34321183  0.33231512 ...  0.21300802  0.7102092
  -0.05771042]
 [-0.28741995  0.31980985 -0.23018652 ...  0.5845511  -0.21329862
   0.72692007]
 [-0.6615692   0.68876815 -0.8743301  ...  0.1087728  -0.26173076
   0.47855455]
 ...
 [-0.22561137 -0.2892573  -0.07064426 ...  0.47566032  0.8327724
   0.40025347]
 [-0.2982421  -0.27473164 -0.05450544 ...  0.4884972   1.0955367
   0.18163365]
 [-0.4437818   0.00930662  0.07223704 ...  0.17290089  1.1833239
   0.07897975]]


## Input layer

In [10]:
text_input_u = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_u')
text_input_v = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_v')

## Pre-processing layer

In [11]:
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing_u')
encoder_input_u = preprocessing_layer(text_input_u)
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing_v')
encoder_input_v = preprocessing_layer(text_input_v)

## BERT encoder layer

In [12]:
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder_u')
outputs_u = encoder(encoder_input_u)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder_v')
outputs_v = encoder(encoder_input_v)

## Pooled output

In [14]:
avg_pool_2d  = tf.keras.layers.AveragePooling2D(pool_size=(128, 1), strides=(128, 1), name='pooling_u')
sequence_output = outputs_u["sequence_output"][0]
sequence_output = tf.reshape(sequence_output, [1, 128, 512, 1])
pooled_u = avg_pool_2d(sequence_output)[0][0]

avg_pool_2d  = tf.keras.layers.AveragePooling2D(pool_size=(128, 1), strides=(128, 1), name='pooling_v')
sequence_output = outputs_v["sequence_output"][0]
sequence_output = tf.reshape(sequence_output, [1, 128, 512, 1])
pooled_v = avg_pool_2d(sequence_output)[0][0]

## Concatenate layer

In [15]:
#concat_layer = tf.keras.layers.Concatenate(name='concat')
#output_concat = concat_layer([pooled_u, pooled_v])

## Output layer

In [16]:
print(pooled_u.shape)
cos_distance = tf.keras.layers.Dot(axes = 1, normalize=True)([tf.reshape(pooled_u, [1, 512, 1]), tf.reshape(pooled_v, [1, 512, 1])]) # where cos (x)
print(cos_distance)
cos_distance = tf.keras.layers.Reshape((1,))(cos_distance)
final = tf.keras.layers.Lambda(lambda x: 1-x)(cos_distance)

#print(output_concat[:,1].shape)

#cosine_loss = tf.keras.losses.CosineSimilarity(axis=-1)
#final = cosine_loss(output_concat[:,0], output_concat[:,1])

print(final.shape)

(512, 1)
KerasTensor(type_spec=TensorSpec(shape=(1, 1, 1), dtype=tf.float32, name=None), name='dot/MatMul:0', description="created by layer 'dot'")
(1, 1)


## Model

In [17]:
model = tf.keras.Model(inputs=[text_input_u, text_input_v], outputs=[final])

tf.keras.utils.plot_model(model)
print(model)

model.summary()

# https://www.sbert.net/docs/training/overview.html

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_u (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 text_v (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 preprocessing_u (KerasLayer)   {'input_word_ids':   0           ['text_u[0][0]']                 
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                      

## Compile

In [18]:
optimizer = tf.optimizers.Adam(0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

df = test.select("sentence1","sentence2","score").toPandas()

print(df.isna().sum())
df = df.fillna('')

#import numpy as np
#df["score"] = np.asarray(df["score"]).astype(np.float32)
#print(list(df["score"]))

results = model.evaluate([df["sentence1"],df["sentence2"]],df["score"])
print("Results: ", results)

sentence1    0
sentence2    3
score        0
dtype: int64
44/44 [==============================] - 16s 311ms/step - loss: 0.2142 - accuracy: 0.0065
Results:  [0.21417468786239624, 0.006526468321681023]


In [19]:
print([df["sentence1"][1],df["sentence1"][1]])
print(df["score"][1])
print(model.predict([df["sentence1"][1:2],df["sentence1"][1:2]]))

['A group of men play soccer on the beach.', 'A group of men play soccer on the beach.']
0.43999996185302725
[[-2.3841858e-07]]


In [20]:
# https://titanwolf.org/Network/Articles/Article?AID=cd44a9bd-d785-4c6e-81a7-8601cc05d636
# https://stackoverflow.com/questions/51003027/computing-cosine-similarity-between-two-tensors-in-keras

## Train

In [ ]:
train.show(5)

In [ ]:
model.fit()

## Prediction

In [ ]:
classifications = model.predict() # unseen data

## Evaluation

In [ ]:
model.evaluate()